## 参考にしたページ

http://www.quuxlabs.com/blog/2010/09/matrix-factorization-a-simple-tutorial-and-implementation-in-python/

http://qiita.com/ysekky/items/c81ff24da0390a74fc6c

https://qiita.com/takechanman/items/6d1f65f94f7aaa016377

## 方策
* CV以外の情報からCVした商品を予測する。

In [2]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import math

# numpyのループを早くしてくれるやつ
import numba

## 小さいデータのサンプル

In [55]:
# np.NANの部分は未知
score = np.array([
        [5, 3, 0, 1],
        [4, 0, 0, 1],
        [1, 1, 0, 5],
        [1, 0, 0, 4],
        [0, 1, 5, 4],
        ]
    )
buy = np.array([
        [1, 0, 0, 1],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 0],
        [0, 1, 1, 0],
        ]
    )

In [56]:
X = pd.DataFrame(score).values
y = pd.DataFrame(buy).values

In [57]:
# 入出力の大きさ
dimension = len(X[0])

## keras

In [58]:
# create regression model
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, BatchNormalization
from sklearn.metrics import mean_squared_error
def reg_model():
    model = Sequential()
    model.add(Dense(1024, input_dim=dimension, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(dimension))

    # compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [60]:
# use data split and fit to run the model
estimator = KerasRegressor(build_fn=reg_model, epochs=100, batch_size=10, verbose=1)
estimator.fit(X, y)
y_pred = estimator.predict(X)

# show its root mean square error
mse = mean_squared_error(y, y_pred)
print("KERAS REG RMSE : %.2f" % (mse ** 0.5))

Epoch 1/100
5/5 [==============================] - 0s - loss: 0.3351
Epoch 2/100
5/5 [==============================] - 0s - loss: 0.0934
Epoch 3/100
5/5 [==============================] - 0s - loss: 0.0599
Epoch 4/100
5/5 [==============================] - 0s - loss: 0.0357
Epoch 5/100
5/5 [==============================] - 0s - loss: 0.0300
Epoch 6/100
5/5 [==============================] - 0s - loss: 0.0274
Epoch 7/100
5/5 [==============================] - 0s - loss: 0.0230
Epoch 8/100
5/5 [==============================] - 0s - loss: 0.0179
Epoch 9/100
5/5 [==============================] - 0s - loss: 0.0162
Epoch 10/100
5/5 [==============================] - 0s - loss: 0.0162
Epoch 11/100
5/5 [==============================] - 0s - loss: 0.0143
Epoch 12/100
5/5 [==============================] - 0s - loss: 0.0105
Epoch 13/100
5/5 [==============================] - 0s - loss: 0.0079
Epoch 14/100
5/5 [==============================] - 0s - loss: 0.0078
Epoch 15/100
5/5 [===========

In [62]:
pd.DataFrame(y)

,0,1,2,3
0,1,0,0,1
1,1,0,0,0
2,0,1,0,0
3,0,0,0,0
4,0,1,1,0


In [63]:
pd.DataFrame(X)

,0,1,2,3
0,5,3,0,1
1,4,0,0,1
2,1,1,0,5
3,1,0,0,4
4,0,1,5,4


In [61]:
# 値が大きい順にそれっぽい予測ができていそうな気がする。
# できていなさそうな気もする。
pd.DataFrame(y_pred)

,0,1,2,3
0,0.999383,0.000046,-0.000629,0.999857
1,0.999945,0.000205,-0.000498,-0.000180
2,0.000572,1.000596,-0.000150,0.000460
3,-0.000222,-0.000363,0.000286,0.001324
4,0.000141,0.999596,1.001289,0.000092


## データの読み込み

In [65]:
file_path = './train/train_A.tsv'
sep = '\t'
target_df = pd.read_csv(file_path, sep)

In [118]:
# コンバージョンは広告経由のみ評価対象とか書いてあってめんどくさい
# ad=1が広告経由のコンバージョン、ad=-1はコンバージョン以外のイベント
score_df = target_df[target_df[u'ad'] == -1]

In [119]:
# ピボットテーブル
# 値の無いところは0で埋める
# カートは評価に影響しない？ので0のままsumする
score  = pd.pivot_table(score_df, values=u'event_type', index=[u'user_id'], columns=[u'product_id'], aggfunc=np.sum, fill_value=0)

In [120]:
# 購買した商品
buy_df = target_df[target_df[u'ad'] == 1]

In [121]:
# scoreをbuyにコピーして、0埋め。購買した商品の部分にフラグを立てる。
buy = score
buy[:] = 0
for b in buy_df.iterrows():
    buy.loc[b[1].user_id,b[1].product_id] += 1

## 実行

In [125]:
X = score.as_matrix()
y = buy.as_matrix()

In [126]:
# 入出力の大きさ
dimension = len(X[0])

In [127]:
dimension

13864

In [128]:
 len(y[0])

13864

In [132]:
# callback
filename = 'machine'
log_filepath = './log_files/{}'.format(filename)

import os

def make_my_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

from keras.callbacks import TensorBoard
tensor_board_path = os.path.join(log_filepath,'tensor_board')
make_my_dir(tensor_board_path)
tb_cb = TensorBoard(log_dir=tensor_board_path, histogram_freq=1)

from keras.callbacks import ModelCheckpoint
weights_path = os.path.join(log_filepath,'weights')
make_my_dir(weights_path)
weights_file_name = os.path.join(weights_path,'weights.{epoch:02d}-{loss:.2f}.hdf5')
cp_cb = ModelCheckpoint(filepath = weights_file_name, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

In [129]:
# create regression model
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, BatchNormalization
from sklearn.metrics import mean_squared_error

np.random.seed(seed=7)

def reg_model():
    model = Sequential()
    model.add(Dense(1024, input_dim=dimension, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(dimension))

    # compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [133]:
# use data split and fit to run the model
estimator = KerasRegressor(build_fn=reg_model, epochs=100, batch_size=dimension/100, verbose=1, callbacks=[tb_cb,cp_cb])
estimator.fit(X, y)
y_pred = estimator.predict(X)

# show its root mean square error
mse = mean_squared_error(y, y_pred)
print("KERAS REG RMSE : %.2f" % (mse ** 0.5))

Epoch 1/100
58650/58658 [============================>.] - ETA: 0s - loss: nan

KeyError: 'acc'

## レコメンド

In [ ]:
# 購買ログにフラグを立てる
buy_log_df = target_df
buy_log_df[u'buy_flg'] = target_df[u'event_type'].apply(lambda x: 1 if x == 3 else 0)
buy_log_pivot_df  = pd.pivot_table(buy_log_df, values=u'buy_flg', index=[u'user_id'], columns=[u'product_id'], aggfunc=np.sum, fill_value=0)

In [ ]:
# 嗜好情報をdfに
# 一部バージョン
ratio_df = pd.DataFrame(nR, index=view_log_pivot_df.index[:10], columns=view_log_pivot_df.columns)

In [ ]:
# レコメンド取得用関数
def get_recommend_items(user_id, ratio_df, buy_log_df, recommend_item_num=22):
    # product_idを対象ユーザのスコア順にソート
    products = ratio_df.loc[user_id].sort_values(ascending=False)
    # 購買履歴のある商品を省く
    buy_log = buy_log_df.loc[user_id]
    # レコメンド対象の商品リスト
    recommend_items = []
    for index in products.index:
        # 購買履歴のある商品は無視
        if buy_log[index] != 0:
            continue
        # 提出時の命名規則でappend
        recommend_items.append([user_id, index, len(recommend_items)])
        # 引数で指定した個数または規程個数(22個)を超えたら終了
        if len(recommend_items) >= recommend_item_num or len(recommend_items) >= 22:
            break
    return recommend_items

In [ ]:
get_recommend_items('0000000_A', ratio_df, buy_log_pivot_df)

## Scikit-learnを使ってNMFを計算する場合
通常のScikit-learnのNMFは欠損値を扱えないので、下記フォークのnmf_missingブランチの内容をインストールする。
`git checkout origin/nmf_missing`
https://github.com/TomDLT/scikit-learn

In [65]:
from sklearn.decomposition import NMF

# R = target_log_pivot_df.as_matrix()[:10]
R = np.array([
        [5, 3, np.NAN, 1],
        [4, np.NAN, np.NAN, 1],
        [1, 1, np.NAN, 5],
        [1, np.NAN, np.NAN, 4],
        [np.NAN, 1, 5, 4],
        ]
    )
k = 2
model = NMF(n_components=k, init='random', random_state=0, solver='mu', max_iter=1000)
P = model.fit_transform(R)
Q = model.components_
print("****************************")
print("k:",k)
print("Pは")
print(P)
print("Q^Tは")
print(Q)
print("P×Q^Tは")
print(np.dot(P,Q))
print("R-P×Q^Tは")
print(model.reconstruction_err_ )

****************************
('k:', 2)
Pは
[[ 0.25533737  2.31920431]
 [ 0.2749493   1.8405892 ]
 [ 1.74751251  0.10992921]
 [ 1.39405084  0.18346339]
 [ 1.39101969  0.25440826]]
Q^Tは
[[ 0.43984129  0.49350833  3.05253861  2.85396988]
 [ 2.10749104  1.23916027  2.96318486  0.11698132]]
P×Q^Tは
[[ 5.00001021  2.99987694  7.65165828  1.00002874]
 [ 3.99995931  2.41647477  6.29329943  1.00001158]
 [ 1.00030298  0.99863188  5.66008999  5.00020773]
 [ 0.99980858  0.91531625  4.79902998  4.00004091]
 [ 1.14799101  1.0017324   5.          3.9996893 ]]
R-P×Q^Tは
0.00227169772504


## 発展？
http://tech-blog.fancs.com/entry/factorization-machines